# Imports
---

In [19]:
from sklearn import svm, linear_model, preprocessing, neighbors
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import cross_val_score

import sklearn
import random
import pandas as pd
import numpy

# Define *Key* class to help build data set
---

In [20]:
class Key:    
    def __init__(self, root, quality = None):
        self.root = root.title()
        self.quality = quality
        self.romans = {}
        
        self.sharps = ["C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B", "C", "C#", "D", "D#", "E", "F", "F#", "G", "G#", "A", "A#", "B", "C"]
        self.flats = ["C", "Db", "D", "Eb", "E", "F", "Gb", "G", "Ab", "A", "Bb", "B", "C", "Db", "D", "Eb", "E", "F", "Gb", "G", "Ab", "A", "Bb", "B", "C"]
        
        self.sharpMajKeys = ["D", "G", "A", "E", "B", "F#", "C#"]
        self.flatMajKeys = ["C", "F", "Bb", "Eb", "Ab", "Db", "Gb", "Cb"]
        self.sharpMinKeys = ["E", "A", "E", "B", "F#", "C#"]
        self.flatMinKeys = ["C", "D", "F", "G", "A"]
        
        self.majorSkips = [2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 1]
        self.minorSkips = [2, 1, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 2]
        
        
    def getMajorScale(self):
        scale = []
        
        # Default cases
        if self.root == "F#":
            scale = ['F#', 'G#', 'A#', 'B', 'C#', 'D#', 'E#', 'F#']
            return scale
        elif self.root == "C#":
            scale = ['C#', 'D#', 'E#', 'F#', 'G#', 'A#', 'B#', 'C#'] 
            return scale
        elif self.root == "Gb":
            scale = ["Gb", "Ab", "Bb", "Cb", "Db", "Eb", "F", "Gb"]
            return scale
        elif self.root == "Cb":
            scale = ["Cb", "Db", "Eb", "Fb", "Gb", "Ab", "Bb", "Cb"]
            return scale
        
        if self.root in self.sharpMajKeys:
            i = self.sharps.index(self.root)
            scaleLength = 0
            skipper = 0
            scale.append(self.sharps[i])

            while scaleLength < 7:
                skipper += self.majorSkips[scaleLength]
                nextNote = self.sharps[i + skipper]

                scale.append(nextNote)
                scaleLength += 1
            return scale
        if self.root in self.flatMajKeys:
            i = self.flats.index(self.root)
            scaleLength = 0
            skipper = 0
            scale.append(self.flats[i])

            while scaleLength < 7:
                skipper += self.majorSkips[scaleLength]
                nextNote = self.flats[i + skipper]

                scale.append(nextNote)
                scaleLength += 1
            return scale

    def getMinorScale(self):
        scale = []
        
        # Default cases
        if self.root == "F#":
            scale = ['F#', 'G#', 'A#', 'B', 'C#', 'D#', 'E#', 'F#']
            return scale
        elif self.root == "C#":
            scale = ['C#', 'D#', 'E#', 'F#', 'G#', 'A#', 'B#', 'C#'] 
            return scale
        elif self.root == "Gb":
            scale = ["Gb", "Ab", "Bb", "Cb", "Db", "Eb", "F", "Gb"]
            return scale
        elif self.root == "Cb":
            scale = ["Cb", "Db", "Eb", "Fb", "Gb", "Ab", "Bb", "Cb"]
            return scale
        
        if self.root in self.sharpMinKeys:
            i = self.sharps.index(self.root)
            scaleLength = 0
            skipper = 0
            scale.append(self.sharps[i])

            while scaleLength < 7:
                skipper += self.minorSkips[scaleLength]
                nextNote = self.sharps[i + skipper]

                scale.append(nextNote)
                scaleLength += 1
            return scale
        if self.root in self.flatMinKeys:
            i = self.flats.index(self.root)
            scaleLength = 0
            skipper = 0
            scale.append(self.flats[i])

            while scaleLength < 7:
                skipper += self.minorSkips[scaleLength]
                nextNote = self.flats[i + skipper]

                scale.append(nextNote)
                scaleLength += 1
            return scale
        
    def getRomans(self):
        romansList = []
        
        if self.quality.lower() == "major":
            romansList = ["I", "ii", "iii", "IV", "V", "vi", "vii°"]
            scale = self.getMajorScale()

        elif self.quality.lower() == "minor":
            romansList = ["i", "ii°", "III", "iv", "v", "VI", "VII"]
            scale = self.getMinorScale()
        
        scale += scale[1:]
        
        for i in range(len(romansList)):
            chord = []
            
            root = 0
            third = 2
            fifth = 4
            seventh = 6
            
            bot = [scale[root+i], random.choice(self.flats+self.sharps)]
            mid = [scale[third+i], random.choice(self.flats+self.sharps)]
            top = [scale[fifth+i], random.choice(self.flats+self.sharps)]
            sev = [scale[seventh + i], "",random.choice(self.flats + self.sharps)]

            prob = [25, 5]
            probSev = [25, 25, 5]

            bot = random.choices(bot, prob)[0]
            mid = random.choices(mid, prob)[0]
            top = random.choices(top, prob)[0]    
            sev = random.choices(sev, probSev)[0]

            
            key = f"{self.root} {self.quality}"
            value = [f"{bot} {mid} {top} {sev}".strip(), romansList[i]]
            
            if self.romans.get(key) is None:
                self.romans[key] = [value]
            else:
                self.romans[key].append(value)


        return self.romans

## Test *Key* class

In [21]:
Key("E", "Minor").getRomans()

{'E Minor': [['E Eb B', 'i'],
  ['F# A C E', 'ii°'],
  ['G B D', 'III'],
  ['A C E', 'iv'],
  ['B D Eb', 'v'],
  ['C A# G', 'VI'],
  ['D F# A', 'VII']]}

# Build data set with random choice of keys
---

In [22]:
sharpMajKeys = ["D", "G", "A", "E", "B", "F#", "C#"]
flatMajKeys = ["C", "F", "Bb", "Eb", "Ab", "Db", "Gb", "Cb"]
sharpMinKeys = ["E", "A", "E", "B"]
flatMinKeys = ["C", "D", "F", "G", "A"]

keys = [sharpMajKeys, flatMajKeys, sharpMinKeys, flatMinKeys]

df = {"Key": [], "Quality": [], "Notes": [], "RomanNumeral": []}

for i in range(1000):
    
    keys = [sharpMajKeys, flatMajKeys, sharpMinKeys, flatMinKeys]
    choiceKeyList = random.choice(keys)
    choiceKey = random.choice(choiceKeyList)
    
    if choiceKey in sharpMajKeys or choiceKey in flatMajKeys:
        choiceQuality = "major"
    if choiceKey in sharpMinKeys or choiceKey in flatMinKeys:
        choiceQuality = "minor"
    
    tempKey = Key(choiceKey, choiceQuality)
    
    for i in range(len(tempKey.getRomans()[f"{tempKey.root} {tempKey.quality}"])): 
        df["Key"].append(tempKey.root)
        df["Quality"].append(tempKey.quality)
        df["Notes"].append(tempKey.getRomans()[f"{tempKey.root} {tempKey.quality}"][i][0])
        df["RomanNumeral"].append(tempKey.getRomans()[f"{tempKey.root} {tempKey.quality}"][i][1])

# Build pandas dataframe
---

In [23]:
df = pd.DataFrame(df)

In [24]:
df.columns

Index(['Key', 'Quality', 'Notes', 'RomanNumeral'], dtype='object')

In [25]:
df.isna().sum()

Key             0
Quality         0
Notes           0
RomanNumeral    0
dtype: int64

In [26]:
df = df #.loc[:, ["Key", "Quality", "Notes", "RomanNumeral"]]

In [27]:
df.head()

,Key,Quality,Notes,RomanNumeral
0,Eb,major,Eb G Bb D,I
1,Eb,major,F Ab C Eb,ii
2,Eb,major,Bb Bb D,iii
3,Eb,major,Ab C Eb G,IV
4,Eb,major,Bb D F,V


# Scikit Learn Classification
---

## Column Transformation and Model

In [28]:
column_trans = make_column_transformer(
(OneHotEncoder(sparse = False), ["Key", "Quality", "RomanNumeral"]), remainder="passthrough")

In [29]:
model = neighbors.KNeighborsClassifier(15)

## Declare Supervised Training Set

In [30]:
X = df.drop(columns=["Notes"])

In [31]:
y = df["Notes"]

### Apply Column Transformation to X

In [32]:
column_trans.fit_transform(X)

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 1., ..., 1., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.]])

## Declare Training and Testing Sets

In [33]:
X_train, y_train, X_test, y_test = train_test_split(X, y, random_state=0, shuffle=True)

## Establish Pipeline to Run Encoding and Model Simultaneously

In [34]:
pipe = make_pipeline(column_trans, model)

In [35]:
X_train.shape, X_test.shape

((5250, 3), (5250,))

## Fit Data to Model

In [36]:
pipe.fit(X, y)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('onehotencoder',
                                                  OneHotEncoder(sparse=False),
                                                  ['Key', 'Quality',
                                                   'RomanNumeral'])])),
                ('kneighborsclassifier', KNeighborsClassifier(n_neighbors=15))])

## Accuracy of Model on X Training Set

In [37]:
pipe.score(X_train, X_test)

0.2895238095238095

## Create Prediction Dataframe

In [38]:
prediction = pipe.predict(X_train)

In [39]:
X_train[:20]

,Key,Quality,RomanNumeral
3838,B,minor,III
5786,A,minor,v
3223,Ab,major,IV
2267,D,minor,VII
4345,D,minor,VI
161,F,minor,i
5548,F#,major,V
1319,Eb,major,IV
3302,F,minor,VI
6388,E,minor,v


In [40]:
prediction[:20]

array(['D F# A', 'E G B', 'Db F Ab', 'C E G Bb', 'Bb D F', 'F Ab C Eb',
       'C# E# G#', 'Ab C Eb', 'Db F Ab C', 'B D F# A', 'Eb G Bb',
       'B D F#', 'Bb D F', 'C Eb G', 'C E G B', 'Eb G Bb', 'G B D',
       'E G Bb', 'Bb Db F Ab', 'Db F Ab C'], dtype=object)

## Predict Random Values

In [41]:
y = pd.DataFrame({"Key": ["F#"], "Quality": ["minor"], "RomanNumeral": ["v"]})

In [42]:
pipe.predict(y)

KeyError: "['Quality'] not in index"